## The LogData class:

In [1]:
import sys
sys.path.append('log-preprocessor')
from LogData import LogData

In [2]:
parser_name = "ApacheAccessParsingModel"

data = LogData(
    data_dir="/home/viktor/projects/aecid/aminer-configuration-engine/data",
    parser_name=parser_name,
)

df = data.get_df()
display(df)

Retrieved previously parsed data.


,/model,/model/client_ip/client_ip,/model/sp1,/model/client_id,/model/sp2,/model/user_id,/model/sp3,/model/time,/model/sp4,/model/fm/request,...,/model/combined/combined/user_agent,/model/combined/combined/sp11,/model/fm/request/method,/model/fm/request/sp5,/model/fm/request/request,/model/fm/request/sp6,/model/fm/request/version,/model/client_ip/localhost,/model/fm/dash,ts
0,10.237.1.238 - - [04/Feb/2022:05:53:37 +0000] ...,10.237.1.238,,-,,-,[,04/Feb/2022:05:53:37 +0000,"] """,GET / HTTP/1.1,...,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:86....,"""",GET,,/,,HTTP/1.1,None,None,2022-02-04 05:53:37+00:00
1,10.237.1.238 - - [04/Feb/2022:05:53:37 +0000] ...,10.237.1.238,,-,,-,[,04/Feb/2022:05:53:37 +0000,"] """,GET /wp-includes/css/dist/block-library/style....,...,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:86....,"""",GET,,/wp-includes/css/dist/block-library/style.min....,,HTTP/1.1,None,None,2022-02-04 05:53:37+00:00
2,10.237.1.238 - - [04/Feb/2022:05:53:37 +0000] ...,10.237.1.238,,-,,-,[,04/Feb/2022:05:53:37 +0000,"] """,GET /wp-content/themes/go/dist/css/design-styl...,...,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:86....,"""",GET,,/wp-content/themes/go/dist/css/design-styles/s...,,HTTP/1.1,None,None,2022-02-04 05:53:37+00:00
3,10.237.1.238 - - [04/Feb/2022:05:53:37 +0000] ...,10.237.1.238,,-,,-,[,04/Feb/2022:05:53:37 +0000,"] """,GET /wp-content/themes/go/dist/js/frontend.min...,...,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:86....,"""",GET,,/wp-content/themes/go/dist/js/frontend.min.js?...,,HTTP/1.1,None,None,2022-02-04 05:53:37+00:00
4,10.237.1.238 - - [04/Feb/2022:05:53:37 +0000] ...,10.237.1.238,,-,,-,[,04/Feb/2022:05:53:37 +0000,"] """,GET /wp-includes/js/wp-emoji-release.min.js?ve...,...,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:86....,"""",GET,,/wp-includes/js/wp-emoji-release.min.js?ver=5.9,,HTTP/1.1,None,None,2022-02-04 05:53:37+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19558,10.237.1.238 - - [08/Feb/2022:06:16:26 +0000] ...,10.237.1.238,,-,,-,[,08/Feb/2022:06:16:26 +0000,"] """,POST /wp-admin/admin-ajax.php HTTP/1.1,...,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:86....,"""",POST,,/wp-admin/admin-ajax.php,,HTTP/1.1,None,None,2022-02-08 06:16:26+00:00
19559,10.237.1.238 - - [08/Feb/2022:06:17:24 +0000] ...,10.237.1.238,,-,,-,[,08/Feb/2022:06:17:24 +0000,"] """,POST /wp-admin/admin-ajax.php HTTP/1.1,...,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:86....,"""",POST,,/wp-admin/admin-ajax.php,,HTTP/1.1,None,None,2022-02-08 06:17:24+00:00
19560,10.237.1.238 - - [08/Feb/2022:06:17:24 +0000] ...,10.237.1.238,,-,,-,[,08/Feb/2022:06:17:24 +0000,"] """,POST /wp-admin/admin-ajax.php HTTP/1.1,...,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:86....,"""",POST,,/wp-admin/admin-ajax.php,,HTTP/1.1,None,None,2022-02-08 06:17:24+00:00
19561,10.237.1.238 - - [08/Feb/2022:06:18:31 +0000] ...,10.237.1.238,,-,,-,[,08/Feb/2022:06:18:31 +0000,"] """,GET / HTTP/1.1,...,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:86....,"""",GET,,/,,HTTP/1.1,None,None,2022-02-08 06:18:31+00:00


## The AMinerModel class:

In [3]:
# necessary for execution in Jupyiter notebook

import getpass
pwd = getpass.getpass("Enter sudo password:")

In [4]:
from tools.AMinerModel import AMinerModel
import yaml

with open("example_config.yml") as file:
    config = yaml.safe_load(file)

model = AMinerModel(config, pwd=pwd)

In [5]:
model.fit_predict(df[:10000], df[10000:])
model.get_latest_results_df()

Training AMiner ...
Finished. (runtime: 5.07883882522583)
Testing AMiner ...
Finished. (runtime: 5.026054859161377)
Raw results saved to: /tmp/aminer_out.json


,detector,feature,idx,timestamp,crit,id
